# Análisis de Inventario y Ventas Históricas

Este notebook analiza los datos de inventario actual contra el histórico de ventas para generar métricas de rendimiento y visualizaciones del inventario.

## Estructura del Análisis
1. Importación de librerías y carga de datos
2. Procesamiento de datos de inventario
3. Procesamiento del histórico de ventas
4. Cálculo de promedios semanales
5. Generación de métricas de rendimiento
6. Tabla resumen
7. Visualización de KPIs

In [15]:
# [1] Importación de librerías y carga de datos
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
from datetime import datetime, timedelta

# Definir rutas de los archivos
data_path = Path('.').resolve().parent / 'data' / 'raw'
inventario_path = data_path / 'INVENTARIO_LOMAROSA.xlsx'
consolidado_path = data_path / 'consolidado.xlsx'

# Cargar datos de inventario (skipear 9 filas)
df_inventario = pd.read_excel(
    inventario_path,
    sheet_name='CONSOLIDADO',  # Asegurarnos de usar la hoja CONSOLIDADO
    skiprows=9
)

# Cargar datos históricos
df_historico = pd.read_excel(
    consolidado_path,
    sheet_name='Sheet1'
)

# Mostrar información básica de los DataFrames
print("=== Información del DataFrame de Inventario ===")
print("Hoja cargada: CONSOLIDADO")
print(df_inventario.info())
print("\n=== Primeras filas del inventario ===")
print(df_inventario.head())
print("\n=== Información del DataFrame Histórico ===")
print(df_historico.info())
print("\n=== Primeras filas del histórico ===")
print(df_historico.head())

=== Información del DataFrame de Inventario ===
Hoja cargada: CONSOLIDADO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Codigo       280 non-null    float64
 1   Productos    280 non-null    object 
 2   Total        282 non-null    float64
 3   U/m          280 non-null    object 
 4   Comentarios  0 non-null      float64
dtypes: float64(3), object(2)
memory usage: 11.1+ KB
None

=== Primeras filas del inventario ===
   Codigo            Productos   Total    U/m  Comentarios
0       3.0              SILLA    0.00  Kilos          NaN
1       4.0  CHULETA DE COGOTE  113.55  Kilos          NaN
2       5.0   MERMA DE PROCESO    0.00  Kilos          NaN
3       6.0   CHULETA DE BRAZO  560.80  Kilos          NaN
4       9.0    CHULETA DE LOMO   80.30  Kilos          NaN

=== Información del DataFrame Histórico ===
<class 'pandas.core.frame.DataFr

In [22]:
# [2] Procesamiento de datos de inventario
print("=== Datos originales de inventario ===")
print("\nPrimeras filas del DataFrame original:")
print(df_inventario.head(10))
print("\nTipos de datos:")
print(df_inventario.dtypes)

# Identificar la columna de código
codigo_columns = [col for col in df_inventario.columns if 'cod' in col.lower()]
if not codigo_columns:
    raise ValueError("No se encontró una columna que contenga 'cod' en su nombre")
codigo_col = codigo_columns[0]
print(f"\nUsando la columna '{codigo_col}' como código de producto")

# Convertir 'Total' a numérico y filtrar valores positivos
df_inventario['Total'] = pd.to_numeric(df_inventario['Total'], errors='coerce')

# Limpiar datos
df_inventario = df_inventario.dropna(subset=[codigo_col, 'Total'])  # Eliminar filas sin código o total
df_inventario = df_inventario[df_inventario['Total'] > 0].copy()  # Solo totales positivos

print("\n=== Datos después de la limpieza inicial ===")
print(df_inventario[[codigo_col, 'Productos', 'Total']].head(10))

# Normalizar los códigos - convertir a enteros
print("\nNormalización de códigos:")
print("Antes:", df_inventario[codigo_col].head(10))

# Convertir a numérico y luego a entero
df_inventario[codigo_col] = pd.to_numeric(df_inventario[codigo_col], errors='coerce')
df_inventario[codigo_col] = df_inventario[codigo_col].astype('Int64')  # Convertir a enteros
df_inventario = df_inventario[df_inventario[codigo_col].notna()]  # Eliminar filas donde el código no es válido

print("Después:", df_inventario[codigo_col].head(10))
print("Tipo de dato:", df_inventario[codigo_col].dtype)

# Seleccionar columnas relevantes
inventario_procesado = df_inventario[[codigo_col, 'Productos', 'Total']].copy()
inventario_procesado = inventario_procesado.rename(columns={
    codigo_col: 'Codigo',
    'Productos': 'Producto',
    'Total': 'Stock_Actual'
})

print("\n=== Resumen de Inventario Procesado ===")
print(f"Total de productos en inventario: {len(inventario_procesado)}")
print("\nMuestra de productos procesados:")
print(inventario_procesado.head(10).to_string())

=== Datos originales de inventario ===

Primeras filas del DataFrame original:
    Codigo             Productos   Total     U/m  Comentarios
1          4   CHULETA DE COGOTE  113.55   Kilos          NaN
3          6    CHULETA DE BRAZO  560.80   Kilos          NaN
4          9     CHULETA DE LOMO   80.30   Kilos          NaN
7         51             CANASTO   45.90   Kilos          NaN
8         52  COSTILLA BABY BACK  436.55   Kilos          NaN
11        55   COSTILLA SAN LUIS  180.45   Kilos          NaN
13        58           COSTIPIEL   75.70   Kilos          NaN
15        61    RACK DE COSTILLA   34.10   Kilos          NaN
16        64  COSTILLA MOSTRADOR    9.10   Kilos          NaN
18        68         CARTILAGOS     6.45  KILOS           NaN

Tipos de datos:
Codigo           Int64
Productos       object
Total          float64
U/m             object
Comentarios    float64
dtype: object

Usando la columna 'Codigo  ' como código de producto

=== Datos después de la limpieza inici

In [23]:
# [3] Procesamiento del histórico de ventas
print("=== Datos originales de ventas ===")
print("\nTipos de documentos únicos en el histórico:")
print(df_historico['Doc'].value_counts())

# Filtrar solo ventas y local PLANTA GALAN
df_historico['Doc'] = df_historico['Doc'].astype(str).str.strip().str.upper()
df_historico['Local'] = df_historico['Local'].astype(str).str.strip().str.upper()

print("\nLocales únicos en el histórico:")
print(df_historico['Local'].unique())

# Filtrar por VENTA y PLANTA GALAN
ventas = df_historico[
    (df_historico['Doc'] == 'VENTA') & 
    (df_historico['Local'] == 'PLANTA GALAN')
].copy()

print(f"\nTotal de registros original: {len(df_historico)}")
print(f"Total de registros después de filtrar por VENTA: {len(df_historico[df_historico['Doc'] == 'VENTA'])}")
print(f"Total de registros después de filtrar por PLANTA GALAN: {len(ventas)}")

# Normalizar códigos y asegurar que sean numéricos
print("\n=== Normalización de códigos ===")
print("Antes de la normalización:")
print(ventas['Cod'].head(10))

# Convertir a numérico y luego a entero
ventas['Cod'] = pd.to_numeric(ventas['Cod'], errors='coerce')
ventas['Cod'] = ventas['Cod'].astype('Int64')  # Usar el mismo tipo que en inventario
ventas = ventas[ventas['Cod'].notna()]

print("\nDespués de la normalización:")
print(ventas['Cod'].head(10))
print("\nTipo de dato de Cod:", ventas['Cod'].dtype)

# Verificar si la columna de fecha existe
fecha_columns = [col for col in ventas.columns if 'fech' in col.lower()]
if not fecha_columns:
    raise ValueError("No se encontró una columna que contenga 'fecha' en su nombre")
fecha_col = fecha_columns[0]
print(f"\nUsando la columna '{fecha_col}' como fecha")

# Seleccionar columnas relevantes usando los nombres correctos
ventas_procesadas = ventas[[fecha_col, 'Cod', 'Kg totales2']].copy()
ventas_procesadas = ventas_procesadas.rename(columns={
    fecha_col: 'fecha',
    'Kg totales2': 'Kg_Vendidos'
})

print("\n=== Resumen de Ventas Procesadas ===")
print(f"Total de registros de venta: {len(ventas_procesadas)}")
print(f"Rango de fechas: {ventas_procesadas['fecha'].min()} a {ventas_procesadas['fecha'].max()}")

# Mostrar ejemplos de ventas por código
print("\nEjemplos de ventas por código:")
ventas_por_codigo = ventas_procesadas.groupby('Cod')['Kg_Vendidos'].agg(['count', 'sum']).reset_index()
print("\nPrimeros 10 productos con sus ventas:")
print(ventas_por_codigo.head(10))

=== Datos originales de ventas ===

Tipos de documentos únicos en el histórico:
Doc
VENTA             124078
TRANSFORMACION     46037
SALIDA             12917
DESPOSTE            6083
TRASLADO            3575
ENTRADA             3025
DEVOLUCION          1846
INVENTARIO          1700
MERMA                483
COMPRAS              314
NOTA CREDITO         127
NOTA                  12
Name: count, dtype: int64

Locales únicos en el histórico:
['PLANTA GALAN' 'LOM' 'NAN' 'M.GALAN' 'OTROS/ OFICINA' 'OTROS']

Total de registros original: 200197
Total de registros después de filtrar por VENTA: 124078
Total de registros después de filtrar por PLANTA GALAN: 12067

=== Normalización de códigos ===
Antes de la normalización:
1      315
2      315
3      315
4      315
203    355
204    208
206    356
219    251
220    251
221    256
Name: Cod, dtype: int64

Después de la normalización:
1      315
2      315
3      315
4      315
203    355
204    208
206    356
219    251
220    251
221    256
Nam

In [24]:
# [4] Cálculo de promedios semanales
print("=== Datos de ventas antes del procesamiento ===")
print(f"Total de registros de venta: {len(ventas_procesadas)}")
print("\nTipo de dato de la columna Cod:", ventas_procesadas['Cod'].dtype)
print("\nEjemplos de códigos en ventas:")
print(ventas_procesadas['Cod'].head(10))

# Calcular el número de semanas en el histórico
fecha_min = ventas_procesadas['fecha'].min()
fecha_max = ventas_procesadas['fecha'].max()
num_semanas = (fecha_max - fecha_min).days / 7

print(f"\nPeríodo analizado: del {fecha_min.strftime('%d/%m/%Y')} al {fecha_max.strftime('%d/%m/%Y')}")
print(f"Total de semanas: {num_semanas:.1f}")

# Calcular totales y promedios por producto
promedios = ventas_procesadas.groupby('Cod').agg({
    'Kg_Vendidos': ['sum', 'count']
}).reset_index()

# Renombrar columnas para claridad
promedios.columns = ['Cod', 'Total_Vendido', 'Num_Ventas']

# Calcular promedio semanal
promedios['Promedio_Semanal'] = promedios['Total_Vendido'] / num_semanas

print("\n=== Datos de promedios calculados ===")
print("Tipo de dato de la columna Cod en promedios:", promedios['Cod'].dtype)
print("\nPrimeras filas de promedios:")
print(promedios.head(10))

# Verificar coincidencias con inventario
print("\n=== Verificación de coincidencias con inventario ===")
print("\nTipo de dato Codigo en inventario:", inventario_procesado['Codigo'].dtype)
print("Ejemplo de códigos en inventario:", inventario_procesado['Codigo'].head(5))
print("\nTipo de dato Cod en promedios:", promedios['Cod'].dtype)
print("Ejemplo de códigos en promedios:", promedios['Cod'].head(5))

productos_en_inventario = set(inventario_procesado['Codigo'])
productos_con_ventas = set(promedios['Cod'])

print(f"\nProductos en inventario: {len(productos_en_inventario)}")
print(f"Productos con historial de ventas: {len(productos_con_ventas)}")
print(f"Productos en común: {len(productos_en_inventario & productos_con_ventas)}")

# Mostrar algunos ejemplos de productos que están en ambos
coincidencias = pd.merge(
    inventario_procesado,
    promedios,
    left_on='Codigo',
    right_on='Cod',
    how='inner'
)

print("\n=== Ejemplos de productos con inventario y ventas ===")
if len(coincidencias) > 0:
    print(coincidencias[['Codigo', 'Producto', 'Stock_Actual', 'Promedio_Semanal']].head(10))
else:
    print("¡No hay coincidencias entre inventario y ventas!")

=== Datos de ventas antes del procesamiento ===
Total de registros de venta: 12067

Tipo de dato de la columna Cod: Int64

Ejemplos de códigos en ventas:
1      315
2      315
3      315
4      315
203    355
204    208
206    356
219    251
220    251
221    256
Name: Cod, dtype: Int64

Período analizado: del 02/01/2025 al 23/09/2025
Total de semanas: 37.7

=== Datos de promedios calculados ===
Tipo de dato de la columna Cod en promedios: Int64

Primeras filas de promedios:
   Cod  Total_Vendido  Num_Ventas  Promedio_Semanal
0    4        363.770          32          9.645417
1    6       7223.878         709        191.542220
2    9        524.850          41         13.916477
3   22         15.460           6          0.409924
4   51       3175.150          58         84.189583
5   52       8993.115         363        238.453807
6   53         12.200           1          0.323485
7   54       1364.110          47         36.169583
8   55      25952.081         455        688.123360


In [25]:
# [5] Análisis de rendimiento de inventario
# Antes de hacer el merge, verificamos los datos
print("=== Verificación de datos antes del merge ===")
print("\nDatos de inventario:")
print("Muestra de códigos en inventario:")
print(inventario_procesado['Codigo'].head())
print("\nDatos de promedios:")
print("Muestra de códigos en ventas:")
print(promedios['Cod'].head())

# Verificar si hay diferencias en el formato de los códigos
print("\n=== Verificación de formatos de código ===")
print("Ejemplo de códigos en inventario:")
for cod in inventario_procesado['Codigo'].head():
    print(f"'{cod}' - tipo: {type(cod)}")
print("\nEjemplo de códigos en ventas:")
for cod in promedios['Cod'].head():
    print(f"'{cod}' - tipo: {type(cod)}")

# Asegurarse de que los códigos estén en el mismo formato
inventario_procesado['Codigo'] = inventario_procesado['Codigo'].astype(str).str.strip().str.upper()
promedios['Cod'] = promedios['Cod'].astype(str).str.strip().str.upper()

# Unir datos de inventario con promedios de venta
analisis = pd.merge(
    inventario_procesado,
    promedios[['Cod', 'Total_Vendido', 'Num_Ventas', 'Promedio_Semanal']],
    left_on='Codigo',
    right_on='Cod',
    how='left'
)

# Mostrar resultados del merge
print("\n=== Resultados del merge ===")
print(f"Registros antes del merge: {len(inventario_procesado)}")
print(f"Registros después del merge: {len(analisis)}")
print("\nEjemplos de registros unidos:")
print(analisis[['Codigo', 'Producto', 'Stock_Actual', 'Promedio_Semanal', 'Total_Vendido', 'Num_Ventas']].head(10))

# Verificar valores nulos
print("\n=== Verificación de valores nulos ===")
print(analisis[['Promedio_Semanal', 'Total_Vendido', 'Num_Ventas']].isnull().sum())

# Rellenar productos sin historial de ventas con promedio 0
analisis['Promedio_Semanal'] = analisis['Promedio_Semanal'].fillna(0)
analisis['Total_Vendido'] = analisis['Total_Vendido'].fillna(0)
analisis['Num_Ventas'] = analisis['Num_Ventas'].fillna(0)

# Calcular métricas
analisis['Estado'] = np.where(
    analisis['Stock_Actual'] >= analisis['Promedio_Semanal'],
    'Stock Adecuado',
    'Bajo Promedio'
)

# Calcular KPIs
total_productos = len(analisis)
productos_stock_adecuado = len(analisis[analisis['Estado'] == 'Stock Adecuado'])
productos_bajo_promedio = len(analisis[analisis['Estado'] == 'Bajo Promedio'])
total_stock_kg = analisis['Stock_Actual'].sum()

print("\n=== Métricas de Rendimiento de Inventario ===")
print(f"Total de productos: {total_productos}")
print(f"Productos con stock adecuado: {productos_stock_adecuado}")
print(f"Productos bajo promedio: {productos_bajo_promedio}")
print(f"Total de stock en kg: {total_stock_kg:,.2f}")

# Mostrar productos con ventas pero sin stock
print("\n=== Productos con ventas pero sin stock adecuado ===")
productos_criticos = analisis[
    (analisis['Promedio_Semanal'] > 0) & 
    (analisis['Stock_Actual'] < analisis['Promedio_Semanal'])
].sort_values('Promedio_Semanal', ascending=False)

print("\nProductos críticos (Top 10):")
print(productos_criticos[['Producto', 'Stock_Actual', 'Promedio_Semanal', 'Total_Vendido']].head(10))

# Guardar métricas para visualización
metricas = {
    'Total Productos': total_productos,
    'Stock Adecuado': productos_stock_adecuado,
    'Bajo Promedio': productos_bajo_promedio,
    'Total Stock (kg)': total_stock_kg
}

# Limpiar el DataFrame final eliminando columnas innecesarias
analisis = analisis.drop('Cod', axis=1)

=== Verificación de datos antes del merge ===

Datos de inventario:
Muestra de códigos en inventario:
1     4
3     6
4     9
7    51
8    52
Name: Codigo, dtype: Int64

Datos de promedios:
Muestra de códigos en ventas:
0     4
1     6
2     9
3    22
4    51
Name: Cod, dtype: Int64

=== Verificación de formatos de código ===
Ejemplo de códigos en inventario:
'4' - tipo: <class 'numpy.int64'>
'6' - tipo: <class 'numpy.int64'>
'9' - tipo: <class 'numpy.int64'>
'51' - tipo: <class 'numpy.int64'>
'52' - tipo: <class 'numpy.int64'>

Ejemplo de códigos en ventas:
'4' - tipo: <class 'numpy.int64'>
'6' - tipo: <class 'numpy.int64'>
'9' - tipo: <class 'numpy.int64'>
'22' - tipo: <class 'numpy.int64'>
'51' - tipo: <class 'numpy.int64'>

=== Resultados del merge ===
Registros antes del merge: 55
Registros después del merge: 55

Ejemplos de registros unidos:
  Codigo            Producto  Stock_Actual  Promedio_Semanal  Total_Vendido  \
0      4   CHULETA DE COGOTE        113.55          9.645417 

In [26]:
# [6] Tabla resumen completa
from IPython.display import display, HTML

# Preparar tabla con toda la información
tabla_resumen = analisis.copy()
tabla_resumen = tabla_resumen.sort_values('Producto')

# Aplicar formato a los números
tabla_resumen['Stock_Actual'] = tabla_resumen['Stock_Actual'].round(2)
tabla_resumen['Promedio_Semanal'] = tabla_resumen['Promedio_Semanal'].round(2)

# Crear estilo HTML para la tabla
tabla_style = """
<style>
    .resumen-table {
        border-collapse: collapse;
        margin: 25px 0;
        font-size: 14px;
        font-family: sans-serif;
        min-width: 400px;
        box-shadow: 0 0 20px rgba(0, 0, 0, 0.15);
    }
    .resumen-table thead tr {
        background-color: #4254b5;
        color: #ffffff;
        text-align: left;
    }
    .resumen-table th,
    .resumen-table td {
        padding: 12px 15px;
    }
    .resumen-table tbody tr {
        border-bottom: 1px solid #dddddd;
    }
    .resumen-table tbody tr:nth-of-type(even) {
        background-color: #f3f3f3;
    }
    .resumen-table tbody tr:last-of-type {
        border-bottom: 2px solid #4254b5;
    }
</style>
"""

# Convertir DataFrame a HTML con estilos
html_table = tabla_resumen.to_html(
    classes=['resumen-table'],
    index=False,
    float_format=lambda x: '{:.2f}'.format(x)
)

html_content = f"""
{tabla_style}
<div style="margin: 20px 0;">
    <h2 style="color: #4254b5; margin-bottom: 10px;">Resumen de Inventario y Promedios de Venta</h2>
    <p style="color: #666; margin-bottom: 20px;">
        Total de productos: <strong>{len(tabla_resumen)}</strong>
    </p>
    {html_table}
</div>
"""

display(HTML(html_content))

# También mostrar en formato texto para fácil copia
print("\nTabla en formato texto:")
print(tabla_resumen.to_string(index=False))

Codigo,Producto,Stock_Actual,Total_Vendido,Num_Ventas,Promedio_Semanal,Estado
365,ASERRIN,156.60,1324.33,20.00,35.11,Stock Adecuado
364,BERIJAS,148.15,6621.39,92.00,175.57,Bajo Promedio
303,BONDIOLA,19.60,20636.35,614.00,547.18,Bajo Promedio
362,BONDIOLA TAJADA,176.60,12741.65,150.00,337.85,Bajo Promedio
333,BRAZO DESPOSTE,210.00,0.00,0.00,0.00,Stock Adecuado
304,BRAZO PULPO,51.68,25162.97,671.00,667.20,Bajo Promedio
251,CABEZA,211.80,31570.17,180.00,837.09,Bajo Promedio
307,CABEZA DE LOMO,761.80,8074.65,244.00,214.10,Stock Adecuado
51,CANASTO,45.90,3175.15,58.00,84.19,Bajo Promedio
305,CARNE MOLIDA,10.20,4533.98,325.00,120.22,Bajo Promedio



Tabla en formato texto:
Codigo                 Producto  Stock_Actual  Total_Vendido  Num_Ventas  Promedio_Semanal         Estado
   365                  ASERRIN        156.60       1324.330        20.0             35.11 Stock Adecuado
   364                  BERIJAS        148.15       6621.390        92.0            175.57  Bajo Promedio
   303                 BONDIOLA         19.60      20636.350       614.0            547.18  Bajo Promedio
   362          BONDIOLA TAJADA        176.60      12741.650       150.0            337.85  Bajo Promedio
   333          BRAZO DESPOSTE         210.00          0.000         0.0              0.00 Stock Adecuado
   304              BRAZO PULPO         51.68      25162.975       671.0            667.20  Bajo Promedio
   251                   CABEZA        211.80      31570.170       180.0            837.09  Bajo Promedio
   307           CABEZA DE LOMO        761.80       8074.655       244.0            214.10 Stock Adecuado
    51               

In [27]:
# [7] Visualización de KPIs con cards
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Crear figura con subplots
fig = make_subplots(
    rows=1, cols=4,
    subplot_titles=(
        "Total Productos",
        "Stock Adecuado",
        "Bajo Stock",
        "Stock Total (kg)"
    ),
    specs=[[{'type': 'domain'}, {'type': 'domain'}, {'type': 'domain'}, {'type': 'domain'}]]
)

# Definir colores
colors = {
    'title': '#2C3E50',  # Azul oscuro para títulos
    'total': '#3498DB',  # Azul para total de productos
    'good': '#2ECC71',   # Verde para stock adecuado
    'bad': '#E74C3C',    # Rojo para bajo stock
    'kg': '#F1C40F'      # Amarillo para total kg
}

# 1. Total de Productos
fig.add_trace(go.Indicator(
    mode="number",
    value=metricas['Total Productos'],
    number={
        "font": {"size": 50, "color": colors['total']},
        "valueformat": ","
    },
    title={
        "text": "Total Productos<br><span style='font-size:0.8em;'>en inventario</span>",
        "font": {"size": 20, "color": colors['title']}
    },
    domain={'row': 0, 'column': 0}
), row=1, col=1)

# 2. Productos con Stock Adecuado
fig.add_trace(go.Indicator(
    mode="number",
    value=metricas['Stock Adecuado'],
    number={
        "font": {"size": 50, "color": colors['good']},
        "valueformat": ","
    },
    title={
        "text": "Stock Adecuado<br><span style='font-size:0.8em;'>sobre el promedio</span>",
        "font": {"size": 20, "color": colors['title']}
    },
    domain={'row': 0, 'column': 1}
), row=1, col=2)

# 3. Productos Bajo Stock
fig.add_trace(go.Indicator(
    mode="number",
    value=metricas['Bajo Promedio'],
    number={
        "font": {"size": 50, "color": colors['bad']},
        "valueformat": ","
    },
    title={
        "text": "Bajo Stock<br><span style='font-size:0.8em;'>bajo el promedio</span>",
        "font": {"size": 20, "color": colors['title']}
    },
    domain={'row': 0, 'column': 2}
), row=1, col=3)

# 4. Total Stock en KG
fig.add_trace(go.Indicator(
    mode="number",
    value=metricas['Total Stock (kg)'],
    number={
        "font": {"size": 50, "color": colors['kg']},
        "valueformat": ",.1f"
    },
    title={
        "text": "Stock Total<br><span style='font-size:0.8em;'>kilogramos</span>",
        "font": {"size": 20, "color": colors['title']}
    },
    domain={'row': 0, 'column': 3}
), row=1, col=4)

# Actualizar el layout
fig.update_layout(
    height=300,
    showlegend=False,
    margin=dict(t=120, b=20, l=20, r=20),
    paper_bgcolor='white',
    plot_bgcolor='white',
    title=dict(
        text="<b>Panel de Control de Inventario</b>",
        x=0.5,
        y=0.95,
        xanchor='center',
        yanchor='top',
        font=dict(size=24, color=colors['title'])
    ),
    grid=dict(rows=1, columns=4, pattern='independent'),
)

# Añadir sombras y bordes a las cards
for annotation in fig.layout.annotations:
    annotation.update(font=dict(size=16, color=colors['title']))

fig.show()

In [28]:
# [8] Alerta de Productos Críticos
from IPython.display import display, HTML

# Contar productos con bajo stock
productos_bajos = len(analisis[analisis['Estado'] == 'Bajo Promedio'])

# Crear HTML para la alerta
alerta_style = """
<style>
    .alerta-critica {
        background-color: #FFEBEE;
        border: 2px solid #E57373;
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .alerta-header {
        display: flex;
        align-items: center;
        margin-bottom: 15px;
        color: #D32F2F;
    }
    .alerta-icono {
        font-size: 24px;
        margin-right: 10px;
    }
    .alerta-titulo {
        font-size: 20px;
        font-weight: bold;
        margin: 0;
    }
    .alerta-mensaje {
        color: #555;
        font-size: 16px;
        margin: 0;
    }
    .productos-criticos {
        margin-top: 15px;
        padding-top: 15px;
        border-top: 1px solid #FFCDD2;
    }
    .producto-item {
        margin: 5px 0;
        color: #555;
    }
</style>
"""

# Calcular el ratio de cobertura (stock actual / promedio semanal)
analisis_criticos = analisis[
    (analisis['Promedio_Semanal'] > 0) & 
    (analisis['Stock_Actual'] < analisis['Promedio_Semanal'])
].copy()
analisis_criticos['Ratio_Cobertura'] = analisis_criticos['Stock_Actual'] / analisis_criticos['Promedio_Semanal']

# Obtener los 5 productos más críticos (menor ratio de cobertura)
productos_criticos = analisis_criticos.sort_values('Ratio_Cobertura').head(5)

# Crear lista de productos críticos en HTML
productos_html = ""
if len(productos_criticos) > 0:
    productos_html = "<div class='productos-criticos'><strong>Productos más críticos:</strong><br>"
    for _, row in productos_criticos.iterrows():
        deficit = row['Promedio_Semanal'] - row['Stock_Actual']
        productos_html += f"<div class='producto-item'>• {row['Producto']}: Stock {row['Stock_Actual']:.1f} kg vs Promedio {row['Promedio_Semanal']:.1f} kg (Déficit: {deficit:.1f} kg)</div>"
    productos_html += "</div>"

alerta_html = f"""
{alerta_style}
<div class="alerta-critica">
    <div class="alerta-header">
        <span class="alerta-icono">⚠️</span>
        <h2 class="alerta-titulo">Alerta Crítica</h2>
    </div>
    <p class="alerta-mensaje">
        Hay <strong>{productos_bajos}</strong> producto(s) con stock por debajo del promedio semanal de ventas.
    </p>
    {productos_html}
</div>
"""

display(HTML(alerta_html))

In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calcular la diferencia entre stock actual y promedio
productos_deficit['Diferencia'] = productos_deficit['Stock_Actual'] - productos_deficit['Promedio_Semanal']

# Obtener los top 5 productos con mayor sobrestock y déficit
top_sobrestock = productos_deficit.nlargest(5, 'Diferencia')[['Producto', 'Stock_Actual', 'Promedio_Semanal']]
top_faltante = productos_deficit.nsmallest(5, 'Diferencia')[['Producto', 'Stock_Actual', 'Promedio_Semanal']]

# Crear figura con dos subplots verticales
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        "Top 5 Productos con Mayor Sobrestock",
        "Top 5 Productos con Mayor Faltante"
    ),
    vertical_spacing=0.2
)

# Colores
colors = {
    'sobrestock': '#E74C3C',    # Rojo para sobrestock
    'stock_normal': '#3498DB',   # Azul para stock normal/promedio
    'deficit': '#E67E22',        # Naranja para déficit
    'text': '#2C3E50'           # Azul oscuro para texto
}

# Función para agregar las trazas con transparencia
def agregar_trazas(datos, row, showlegend=True):
    # Primero agregamos el Stock Actual
    for producto in datos['Producto']:
        d = datos[datos['Producto'] == producto]
        color = colors['sobrestock'] if d['Stock_Actual'].iloc[0] > d['Promedio_Semanal'].iloc[0] else colors['deficit']
        fig.add_trace(
            go.Bar(
                name='Stock Actual',
                x=[producto],
                y=[d['Stock_Actual'].iloc[0]],
                marker=dict(
                    color=color,
                    opacity=0.7  # Añadimos transparencia
                ),
                showlegend=(showlegend and producto == datos['Producto'].iloc[0])
            ),
            row=row, col=1
        )
    
    # Luego agregamos el Promedio Semanal como línea con marcadores
    fig.add_trace(
        go.Scatter(
            name='Promedio Semanal',
            x=datos['Producto'],
            y=datos['Promedio_Semanal'],
            mode='lines+markers',
            line=dict(
                color=colors['stock_normal'],
                width=3
            ),
            marker=dict(
                size=10,
                symbol='diamond'
            ),
            showlegend=showlegend
        ),
        row=row, col=1
    )

# Agregar las trazas para ambos subplots
agregar_trazas(top_sobrestock, row=1, showlegend=True)
agregar_trazas(top_faltante, row=2, showlegend=False)

# Actualizar el diseño
fig.update_layout(
    height=800,
    template='plotly_white',
    barmode='overlay',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    font=dict(
        family="Arial",
        size=12,
        color=colors['text']
    )
)

# Actualizar los ejes y agregar títulos
for i in [1, 2]:
    fig.update_xaxes(title_text="Producto", row=i, col=1)
    fig.update_yaxes(title_text="Cantidad", row=i, col=1)

fig.show()

NameError: name 'productos_deficit' is not defined

In [30]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Distribución del Stock vs Promedio
fig_distribucion = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Top 10 Productos con Mayor Sobrestock",
        "Top 10 Productos con Mayor Faltante",
        "Distribución del Estado de Inventario",
        "Productos con Mayor Rotación"
    ),
    vertical_spacing=0.16,
    horizontal_spacing=0.1,
    specs=[
        [{"type": "bar"}, {"type": "bar"}],
        [{"type": "pie"}, {"type": "bar"}]
    ]
)

# Colores consistentes para todo el dashboard
colors = {
    'sobrestock': '#E74C3C',    # Rojo para sobrestock
    'stock_normal': '#3498DB',   # Azul para stock normal
    'deficit': '#E67E22',       # Naranja para déficit
    'text': '#2C3E50',         # Azul oscuro para texto
    'background': '#ECF0F1'    # Gris claro para fondo
}

# Preparar datos
analisis['Diferencia'] = analisis['Stock_Actual'] - analisis['Promedio_Semanal']
top_sobrestock = analisis.nlargest(10, 'Diferencia')
top_deficit = analisis.nsmallest(10, 'Diferencia')

# 1. Gráfico de Sobrestock (Top 10)
for producto in top_sobrestock['Producto']:
    datos = top_sobrestock[top_sobrestock['Producto'] == producto]
    # Barra de Stock Actual
    fig_distribucion.add_trace(
        go.Bar(
            name='Stock Actual',
            x=[producto],
            y=[datos['Stock_Actual'].iloc[0]],
            marker_color=colors['sobrestock'],
            opacity=0.7,
            showlegend=(producto == top_sobrestock['Producto'].iloc[0])
        ),
        row=1, col=1
    )
    # Línea de Promedio
    fig_distribucion.add_trace(
        go.Scatter(
            name='Promedio Semanal',
            x=[producto],
            y=[datos['Promedio_Semanal'].iloc[0]],
            mode='markers',
            marker=dict(
                symbol='diamond',
                size=10,
                color=colors['stock_normal'],
                line=dict(width=2, color='white')
            ),
            showlegend=(producto == top_sobrestock['Producto'].iloc[0])
        ),
        row=1, col=1
    )

# 2. Gráfico de Déficit (Top 10)
for producto in top_deficit['Producto']:
    datos = top_deficit[top_deficit['Producto'] == producto]
    # Barra de Stock Actual
    fig_distribucion.add_trace(
        go.Bar(
            name='Stock Actual',
            x=[producto],
            y=[datos['Stock_Actual'].iloc[0]],
            marker_color=colors['deficit'],
            opacity=0.7,
            showlegend=False
        ),
        row=1, col=2
    )
    # Línea de Promedio
    fig_distribucion.add_trace(
        go.Scatter(
            name='Promedio Semanal',
            x=[producto],
            y=[datos['Promedio_Semanal'].iloc[0]],
            mode='markers',
            marker=dict(
                symbol='diamond',
                size=10,
                color=colors['stock_normal'],
                line=dict(width=2, color='white')
            ),
            showlegend=False
        ),
        row=1, col=2
    )

# 3. Gráfico de Distribución (Pie)
estados_conteo = analisis['Estado'].value_counts()
fig_distribucion.add_trace(
    go.Pie(
        labels=estados_conteo.index,
        values=estados_conteo.values,
        hole=0.4,
        marker_colors=[colors['sobrestock'], colors['stock_normal']],
        textinfo='percent+label',
        textposition='outside',
        showlegend=False
    ),
    row=2, col=1
)

# 4. Top 10 Productos con Mayor Rotación
top_rotacion = analisis.nlargest(10, 'Num_Ventas')
fig_distribucion.add_trace(
    go.Bar(
        x=top_rotacion['Producto'],
        y=top_rotacion['Num_Ventas'],
        marker_color=colors['stock_normal'],
        name='Número de Ventas',
        showlegend=False
    ),
    row=2, col=2
)

# Actualizar layout y formato
fig_distribucion.update_layout(
    title=dict(
        text='<b>Dashboard de Control de Inventario</b>',
        x=0.5,
        y=0.98,
        xanchor='center',
        yanchor='top',
        font=dict(size=24, color=colors['text'])
    ),
    height=1000,
    showlegend=True,
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    font=dict(family="Arial", size=12, color=colors['text'])
)

# Actualizar ejes y formato para cada subplot
fig_distribucion.update_xaxes(tickangle=45, row=1, col=1)
fig_distribucion.update_xaxes(tickangle=45, row=1, col=2)
fig_distribucion.update_xaxes(tickangle=45, row=2, col=2)

# Agregar títulos a los ejes
fig_distribucion.update_xaxes(title_text="Producto", row=1, col=1)
fig_distribucion.update_yaxes(title_text="Cantidad (kg)", row=1, col=1)
fig_distribucion.update_xaxes(title_text="Producto", row=1, col=2)
fig_distribucion.update_yaxes(title_text="Cantidad (kg)", row=1, col=2)
fig_distribucion.update_xaxes(title_text="Producto", row=2, col=2)
fig_distribucion.update_yaxes(title_text="Número de Ventas", row=2, col=2)

# Ajustar márgenes
fig_distribucion.update_layout(margin=dict(t=120, b=20, l=20, r=20))

fig_distribucion.show()

In [31]:
# Crear un dashboard con métricas clave y recomendaciones
from IPython.display import display, HTML

# Calcular métricas adicionales
total_productos = len(analisis)
productos_sin_ventas = len(analisis[analisis['Num_Ventas'] == 0])
productos_criticos = len(analisis[
    (analisis['Stock_Actual'] < analisis['Promedio_Semanal']) & 
    (analisis['Promedio_Semanal'] > 0)
])

# Calcular porcentajes
pct_sin_ventas = (productos_sin_ventas / total_productos) * 100
pct_criticos = (productos_criticos / total_productos) * 100

# Estilo CSS para el dashboard
dashboard_style = """
<style>
    .dashboard-container {
        font-family: Arial, sans-serif;
        max-width: 1200px;
        margin: 20px auto;
        padding: 20px;
    }
    .metric-grid {
        display: grid;
        grid-template-columns: repeat(3, 1fr);
        gap: 20px;
        margin-bottom: 30px;
    }
    .metric-card {
        background: white;
        border-radius: 10px;
        padding: 20px;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .metric-title {
        color: #2C3E50;
        font-size: 16px;
        margin-bottom: 10px;
    }
    .metric-value {
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 5px;
    }
    .metric-description {
        font-size: 14px;
        color: #666;
    }
    .recommendations {
        background: #F8F9FA;
        border-radius: 10px;
        padding: 20px;
        margin-top: 20px;
    }
    .status-good { color: #2ECC71; }
    .status-warning { color: #F1C40F; }
    .status-critical { color: #E74C3C; }
</style>
"""

# Crear el contenido HTML
dashboard_html = f"""
{dashboard_style}
<div class="dashboard-container">
    <h2 style="color: #2C3E50; margin-bottom: 20px;">Resumen Ejecutivo de Inventario</h2>
    
    <div class="metric-grid">
        <div class="metric-card">
            <div class="metric-title">Productos Sin Movimiento</div>
            <div class="metric-value status-warning">{productos_sin_ventas:,}</div>
            <div class="metric-description">
                {pct_sin_ventas:.1f}% del inventario no ha tenido ventas en el período analizado
            </div>
        </div>
        
        <div class="metric-card">
            <div class="metric-title">Productos en Estado Crítico</div>
            <div class="metric-value status-critical">{productos_criticos:,}</div>
            <div class="metric-description">
                {pct_criticos:.1f}% de los productos tienen stock por debajo del promedio de ventas
            </div>
        </div>
        
        <div class="metric-card">
            <div class="metric-title">Stock Total</div>
            <div class="metric-value status-good">{analisis['Stock_Actual'].sum():,.0f} kg</div>
            <div class="metric-description">
                Distribuido en {total_productos:,} productos diferentes
            </div>
        </div>
    </div>
    
    <div class="recommendations">
        <h3 style="color: #2C3E50; margin-bottom: 15px;">Recomendaciones Principales</h3>
        <ul style="color: #555;">
            <li><strong>Atención Inmediata:</strong> {len(top_deficit):,} productos requieren reposición urgente.</li>
            <li><strong>Sobrestock:</strong> {len(top_sobrestock):,} productos tienen niveles de stock significativamente altos.</li>
            <li><strong>Productos Sin Movimiento:</strong> Evaluar estrategias para {productos_sin_ventas:,} productos sin ventas recientes.</li>
        </ul>
    </div>
</div>
"""

display(HTML(dashboard_html))

# Mostrar tabla de productos críticos que requieren atención inmediata
print("\nProductos que Requieren Atención Inmediata:")
productos_criticos_df = analisis[
    (analisis['Stock_Actual'] < analisis['Promedio_Semanal']) & 
    (analisis['Promedio_Semanal'] > 0)
].sort_values('Diferencia')[['Producto', 'Stock_Actual', 'Promedio_Semanal', 'Num_Ventas']].head(10)

print("\nTop 10 Productos Críticos:")
print(productos_criticos_df.to_string(index=False))


Productos que Requieren Atención Inmediata:

Top 10 Productos Críticos:
          Producto  Stock_Actual  Promedio_Semanal  Num_Ventas
MILANESA DE PIERNA        689.65       3248.349470       268.0
            CABEZA        211.80        837.087841       180.0
       BRAZO PULPO         51.68        667.200095       671.0
     LOMO ALMENDRA        179.25        778.785795       829.0
  TOCINO CORRIENTE        111.40        687.672841       233.0
          BONDIOLA         19.60        547.175947       614.0
 COSTILLA SAN LUIS        180.45        688.123360       455.0
      HUESO POROZO         54.80        537.013087       156.0
     TOCINO DORSAL         68.25        438.980871       104.0
           PLANCHA        163.85        532.513174       881.0
